# 使用timescaledb支持时序数据库

[timescaledb](https://docs.timescale.com/timescaledb/latest/)是一个开源的pg时序数据库插件,它的主要特性是:

1. 专用的表类型`超级表(hyper table)`和`分布式超级表(distributed hypertable)`用于专门处理时序数据
2. 专门针对时序数据的分表方案
3. 专门针对时序数据的冷数据压缩和存储方案
4. 连续聚合函数用于处理流数据
5. 自动过期方案

这个插件经过索引优化和并行处理后查询性能很高,完全可以满足多数场景下对时序数据库的期待.